# Text Generation with Markov Chains:
This notebook explores the idea of markov chains by following the classic example of text generation. 

The code I wirte here is heavily based on this video tutorial: https://www.youtube.com/watch?v=eGFJ8vugIWA

## What is a Markov chain?

Below is a summary of my understandin of Markov Chains. 

For a more thorough (and more accurate) description see this wikipedia page. Also the applications section is very interesting: 

https://en.wikipedia.org/wiki/Markov_chain

Also having previosly read this book chapter helped me more quickly grasp the Markov chain process. I find it easiest to think of the Markov chain as a probablistic state machine. 

https://gameprogrammingpatterns.com/state.html
 

The Markov chain is constructed of a set of $N$ possible states and a set of $K$ transistions between states. At any given time the system is in one state and has some *pre-defined* probabily of transitioning to another state. It is possible to transtion from any one state to the any other state (includeing transitioning back to the same state). So if the sytem is in state $n$ it has $K$ possible transitions with probabilities $p_{n,k}$ (probability of transitioning from state $n$ to state $k$). For each state the sum of all transition probabilites must be equal to 1. Also I chose to think of each state having a transition to all other states, but there is nothing preventing $p_{nk}=0$ which would imply it is actually impossible to make that transition. 

Mathametically we can consider a markov chain using linear algebra. We can model the states as a column vector $\vec{s}$, and the transitions as a matrix $T$. Each column of $T$ represents the possible transtion probabilties for the correspoding row of $\vec{s}$. To predict the probabiltied of which state the system will be in after $n$ transititons we can use the following equation:

$$
\vec{s}_n = T^n\vec{s}_0
$$

## How to use this Model to generate Text?

It turns out we can model plain text as a markov chain. In its simplest form we can think of text as a sequence of characters. Then this could be viewed as a markov chain where each character in a language is a possible state that has some probability of moving to another character. Alternitively we can see each full word as a state and transition to the next word. 

Or every $n$ characters is a state. This is what is commenly reffered to as an n-gram, and it is commonly to use this model. 

To Generate text with a Markov Chain we first need to construct the chain itself, and then use that chain/model to generate new text. You could technically generate the markov chain manually, but instead it is more accurate and efficent to generate the model based on some input source text. 

## The Code

The Code below shows one way to gnerate a markov model based on an input source text. This code is basically identical to the code explained in this video I linked. I just cleaned it up a bit and wrapped it into a function. We use a dictionary to model our markov chain where each key in the dictionary is a state and each value is a list of transitions. 

I loop through each charater in the source text and generate the current state/gram. 

If the gram/state does not already exist in our model then we add it to the model. 

Then I append the single character that appears after the currnet gram to its transition list. Note it is possible to have recurring characters in our transtion list, but this actually bakes in the probabilities for us during the generation, as we will see more clearly later.

For the last gram in the text I append the first charcter in the text to the transition list and then break the for loop. 

Finally we return the model. 

In [3]:
import random
import pprint
pp = pprint.pprint

In [4]:
def gen_markov_model(source:str, order:int)->dict:
    model = {}
    for i in range(0,len(source)):
        #print('----------\n{}:{}'.format(i+order, len(source)))
        eidx = i+order
        gram = source_text[i:eidx:1]
        #print(gram)
        #if this gram is new add it as a key
        if not gram in model.keys():
            model[gram] = []
        if eidx < len(source):
            next_char = source_text[i+order]
            model[gram].append(next_char)
        else:
            next_char = source_text[0]
            model[gram].append(next_char)
            break
        #print('{} -> {}'.format(gram, next_char))
    return model

In [5]:
source_text = 'hello my name is joey. hello my name is josh. hello my name is john'
order = 3
x = gen_markov_model(source_text, order)

L = len(source_text) 
keys = list(x.keys())
last_n_gram = source_text[L-order:L]

print(len(x))
pp(x)
print(last_n_gram in keys)

29
{' he': ['l', 'l'],
 ' is': [' ', ' ', ' '],
 ' jo': ['e', 's', 'h'],
 ' my': [' ', ' ', ' '],
 ' na': ['m', 'm', 'm'],
 '. h': ['e', 'e'],
 'ame': [' ', ' ', ' '],
 'e i': ['s', 's', 's'],
 'ell': ['o', 'o', 'o'],
 'ey.': [' '],
 'h. ': ['h'],
 'hel': ['l', 'l', 'l'],
 'is ': ['j', 'j', 'j'],
 'joe': ['y'],
 'joh': ['n'],
 'jos': ['h'],
 'llo': [' ', ' ', ' '],
 'lo ': ['m', 'm', 'm'],
 'me ': ['i', 'i', 'i'],
 'my ': ['n', 'n', 'n'],
 'nam': ['e', 'e', 'e'],
 'o m': ['y', 'y', 'y'],
 'oey': ['.'],
 'ohn': ['h'],
 'osh': ['.'],
 's j': ['o', 'o', 'o'],
 'sh.': [' '],
 'y n': ['a', 'a', 'a'],
 'y. ': ['h']}
True


## Using the Model to Generate Text

Now that we can generate a model, we can use it!

The function below generates text based on a given markov model. 

We tell the model which state to start in and how many characters to generate. 

Then we simply run a loop where we randomly choose the next charater from the current states transition list. Sine our model just added duplicate charaters we can just choose from the list by weighing each elemet with an equal probability. 

We add the nexxt character to the result and defin the current state as the last n charcters of the result before returning to the top of the loop. 

In [6]:
def generate_markov_text(start_gram:str, model:dict, model_order:int, gen_len:int)->str:
    if start_gram not in model.keys():
        print('start gram must appear in the model keys: {}'.format(model.keys()))
        return 
    result = start_gram
    cur_gram = start_gram
    for i in range(0,gen_len):
        if cur_gram not in model.keys(): break #stop from crashing on edge case 
        next_char = random.choice(model[cur_gram])
        result += next_char
        cur_gram = result[len(result)-model_order:len(result):1]
    return result
    

Here I copied a paragraph from wikipedia that discussed the Toledo War. The model gets more beliveable the more input text you give it, and interestingly will mimic the style of the sou

In [7]:
source_text = 'The Toledo War (1835–36), also known as the Michigan–Ohio War or the Ohio–Michigan War, was a boundary dispute between the U.S. state of Ohio and the adjoining territory of Michigan over what is now known as the Toledo Strip. Control of the mouth of the Maumee River and the inland shipping opportunities it represented, and the good farmland to the west were seen by both parties as valuable economic assets. Poor geographical understanding of the Great Lakes helped produce conflicting state and federal legislation between 1787 and 1805, and varying interpretations of the laws led the governments of Ohio and Michigan to both claim jurisdiction over a 468-square-mile (1,210 km2) region along their border. The situation came to a head when Michigan petitioned for statehood in 1835 and sought to include the disputed territory within its boundaries. Both sides passed legislation attempting to force the other sides capitulation, and Ohios Governor Robert Lucas and Michigans 24-year-old Boy Governor Stevens T. Mason helped institute criminal penalties for residents submitting to the others authority. Both states deployed militias on opposite sides of the Maumee River near Toledo, but besides mutual taunting, there was little interaction between the two forces. The single military confrontation of the war ended with a report of shots being fired into the air, incurring no casualties. The only blood spilled was the non-fatal stabbing of a law enforcement officer. During the summer of 1836, the United States Congress proposed a compromise whereby Michigan gave up its claim to the strip in exchange for its statehood and the remaining three-quarters of the Upper Peninsula. The northern regions mineral wealth later became an economic asset to Michigan, but at the time the compromise was considered a poor deal for the new state, and voters in a statehood convention in September soundly rejected it. But in December, facing a dire financial crisis and pressure from Congress and President Andrew Jackson, the Michigan government called another convention (called the Frostbitten Convention), which accepted the compromise, resolving the Toledo War. '

In [8]:
order = 4
model = gen_markov_model(source_text, order)
start_gram = source_text[0:order:1]
N = 1000

In [9]:
random_text = generate_markov_text(start_gram, model, order, N)

In [168]:
print(random_text)

The only blood another sides capitulation opposite sident Andrew Jackson, the northers authority. Both sides passet to the summer of the only blood and to a head whereby Michigan–Ohio War (1835 and soundaries. Both claim jurisdictioned for statehood spilled and the Maumee River near Toledo Strip. Convention), which acceptember sides of shots being of Michigan over and the good spilled with a repress produce compromise was little inland the west were seen by both parties. The non-fatal states deployed military confrontation), which accepted States deployed militias on over and the disputed territory of the Maumee River a 468-square-mile (1,210 km2) regions of a law enforces. Both parties. Both states Congress and Michigan–Ohio and shipping of Michigan governor Robert Lucas and Michigan gover and the Upper Peninsula. The situation opport of the U.S. stabbing the northereby Michigan to the Michigans 24-year-old Boy Governor Robert Lucas a boundly rejected the mouth of a law enforcement of